In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
import rarfile, os

def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])


In [ ]:
img = rgb2gray(plt.imread('test.jpg'))
img.shape

In [ ]:
plt.figure(figsize=(12,6))
plt.imshow(img, cmap=plt.cm.gray)

In [ ]:
def plotBox(xmin, xmax, ymin, ymax):
    plt.plot([xmin, xmax, xmax, xmin, xmin],
             [ymin, ymin, ymax, ymax, ymin], color='r', lw=2)

def threshold(img, thresh=60):
    Y = np.zeros(img.shape)+255
    Y[img < thresh] = img[img < thresh]
    return(Y)
    
# linethresh = 180

nx,ny = 90,90
Y = threshold(img)
linethresh = np.clip(np.percentile(np.mean(Y, axis=1), 4), 0, 180)
ylines = np.where(np.mean(Y,1) <= linethresh)[0]
if len(ylines):
    # two rows
    lines = (np.concatenate([[0], ylines,[img.shape[0]]]))
    deltas = np.diff(lines)
    yoffsets = lines[np.where(deltas>=ny)[0]]#[::2]
    heights = deltas[np.where(deltas>=ny)[0]]#[::2]
    print(lines)
    print(deltas)
    print (yoffsets)
    print(heights)
else:
    print(ylines)
    plt.plot(np.mean(Y,1))
    raise(ValueError('Could not figure out boxes'))

plt.figure(figsize=(12,6))
plt.imshow(img, cmap=plt.cm.gray)
plt.xlim(-1, img.shape[1]+1)
plt.ylim(img.shape[0]+1,-1)
    
for yoffset,height in zip(yoffsets, heights):
    X = Y[yoffset:yoffset+height,:]
    linethresh = np.clip(np.percentile(np.mean(X, axis=0), 1),0,180)
    xlines = np.where(np.mean(X, 0) <= linethresh)[0]
    
    if len(xlines):
        # 4 columns
        lines = (np.concatenate([[0], xlines,[img.shape[1]]]))
        deltas = np.diff(lines)
        xoffsets = lines[np.where(deltas>=nx)[0]]#[::2]
        widths = deltas[np.where(deltas>=nx)[0]]#[::2]
        print(linethresh, len(xlines))
        for xoffset, width in zip(xoffsets, widths):
            plotBox(xoffset, xoffset+width, yoffset, yoffset+height)
    else:
        print(len(xlines))
        raise(ValueError('Not Sure'))
#         break
    
#     break

# plt.figure(figsize=(12,6))
# plt.imshow(Y, cmap=plt.cm.gray)
plt.figure(figsize=(6,6))
plt.plot(np.mean(Y, axis=1))
    
# plt.figure(figsize=(12,6))
# plt.imshow(X, cmap=plt.cm.gray)
# plt.figure(figsize=(6,6))
# plt.plot(np.mean(X, axis=0))


# plt.plot(np.mean(X, axis=0))

In [ ]:
def carve(img, nx=100, ny=100, linethresh=None):
    Y = threshold(img)
    linethresh = np.clip(np.percentile(np.mean(Y, axis=1), 4), 0, 180)
    ylines = np.where(np.mean(Y,1)<linethresh)[0]
    lines = (np.concatenate([[0], ylines,[img.shape[0]]]))
    deltas = np.diff(lines)
    yoffsets = lines[np.where(deltas>ny)[0]]#[::2]
    heights = deltas[np.where(deltas>ny)[0]]#[::2]
    
    for yoffset,height in zip(yoffsets, heights):
        X = Y[yoffset:yoffset+height,:]
        xmean = np.mean(X, axis=0)/np.std(X, axis=0)
        xlines = np.where(xmean < 1)[0]
#         linethresh = np.clip(np.percentile(xmean, 4), 0, 180)
#         linethresh = np.percentile(xmean, np.pi)
#         xlines = np.where(np.mean(X, 0) < linethresh)[0]

        lines = (np.concatenate([[0], xlines,[img.shape[1]]]))
        deltas = np.diff(lines)
        xoffsets = lines[np.where(deltas>ny)[0]]#[::2]
        widths = deltas[np.where(deltas>ny)[0]]#[::2]
        
#         plt.figure(1)
#         plt.axhline(1)
#         plt.plot(xmean)
#         plt.xlim(0,2)
        
#         print(sxoffsets)
#         print(widths)
        for xoffset, width in zip(xoffsets, widths):
            yield xoffset, xoffset+width, yoffset, yoffset+height
#             yield img[yoffset:yoffset+height, 
#                       xoffset:xoffset+width]

plt.figure(0, figsize=(12,6))
pattern = '/Users/ajmendez/data/dilbert/raw/*.rar'
for filename in sorted(glob.iglob(pattern)):
    try:
        with rarfile.RarFile(filename, crc_check=False) as rf:
            files = sorted(rf.namelist())
            goodfiles = [f for f in files
                         if ('.gif' in f) or (not (f.replace('.jpg','-colour.jpg') in files) )]
            for i,f in enumerate(goodfiles):
                img = rgb2gray(plt.imread(rf.open(f)))
                plt.figure('{} {}'.format(filename,i))
                plt.imshow(img, cmap=plt.cm.gray)
                for im in carve(img):
                    plotBox(*im)
                plt.autoscale(enable=True, axis='both', tight=True)
                if i > 1:
                    break
    except Exception as e:
        print('Failed to load: {}'.format(filename))
#             break
#     break
plt.show()